In [ ]:
import pandas as pd
import torch

from model.encoder import Encoder

from utils.misc import cosine_distance

In [2]:
movies = pd.read_csv("data/ml-20m/movies.csv", header=0, names=["movie_id", "movie_title", "genres"])

movies = movies[["movie_id", "movie_title"]]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

movie_embeddings = torch.load("weights/recommender/deepfm.pt", map_location="cpu")["embedding.embedding.weight"][943:]

encoder = Encoder().to(device)

encoder.load_state_dict(torch.load("weights/encoder/encoder.pt", map_location=device))

In [ ]:
request = "I want to watch a romantic comedy."

request_embedding = encoder(request).cpu()

distances = cosine_distance(request_embedding, movie_embeddings)

_, indices = torch.topk(distances, k=10, largest=False)

movies.iloc[indices]